In [ ]:
import TrainModel as Train
import PredictData as Predict

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
import umap


In [ ]:
def CheckNan(df):
    nan_columns = df.isna().sum()

    if len(nan_columns[nan_columns> 0]) >0:
        print(f"{len(df)} students")
        raise ValueError(f"These columns contain NaN values this is not going work! \n {nan_columns[nan_columns> 0]}")
    
    print(f"All good")
    return

    
def RemoveEntries(df):

    df2 = df

    _base = len(df)
    print()
    #df = df[df["Credits-Y1"] > 0].reset_index(drop=True)
    df = df.dropna(subset=["OverAll"]).reset_index(drop=True)
    df = df.dropna(subset=["B1B2"]).reset_index(drop=True)
    

        
    diff_df = df2.merge(df,on=list(df2.columns), how = 'left', indicator=True)
    missing_rows_df = diff_df[diff_df["_merge"] == "left_only"].drop(columns=["_merge"])

    print(f"Started with {_base} students")
    print(f"dropped {_base-len(df)} students")
    print(missing_rows_df["BSA"].value_counts())
    print(f"\n =======Remaining========= \n {len(df)}")
    print(df["BSA"].value_counts())
    return df


def TheQuiters(merged_df):
    columns_to_check = [col for col in merged_df.columns if col.startswith("x_Course") and col.endswith("Both_notPresent")]

    rows_to_remove = merged_df[merged_df[columns_to_check].sum(axis=1) >= 3].index
    print(len(rows_to_remove))
    merged_df = merged_df.drop(rows_to_remove)
    return merged_df

def PrepCluster(df, UseNationalities= False, UsePreEducation =True):
    df['Gender'] = df['Gender'].apply(lambda x: 1 if x == 'F' else 0)

    columns_to_drop = ['train', 'Year', 'BSA', 'Crd-B1B2', "Program"]

    df_Clustering = df.drop(columns=columns_to_drop)

    #Make Nationalities dummies
    if UseNationalities:
        #Create dummy variables
        nationality_dummies = pd.get_dummies(df_Clustering['Nationality'], prefix='Nationality',dtype='int')

        if "Nationality_Niet toeg. Nationaliteit(n/e)" in nationality_dummies.columns:
            nationality_dummies = nationality_dummies.drop(["Nationality_Niet toeg. Nationaliteit(n/e)"], axis=1)
            print('Dropped Nationalities')
            
        # Concatenate the dummy variables with the original DataFrame
        df_Clustering = pd.concat([df_Clustering, nationality_dummies], axis=1)

        Nationalities= ['Nationality_Afrika', 'Nationality_Azie', 'Nationality_EU',
        'Nationality_Europa', 'Nationality_Mid-Zuid-Amerika',
        'Nationality_Nederland', 'Nationality_Noord-Amerika',
        'Nationality_Oceanie', 'Nationality_Onbekend']

        for col_name in Nationalities:
            if col_name not in df_Clustering:
                df_Clustering[col_name] = 0

    if UsePreEducation:
        #Create dummy variables
        preEducation_dummies = pd.get_dummies(df_Clustering['PreEducation'], prefix='PreEducation',dtype='int')

        if "PreEducation_nvt" in preEducation_dummies.columns:
            preEducation_dummies = preEducation_dummies.drop(["PreEducation_nvt"], axis=1)
            print('Dropped PreEducation_Overig')
            
        if "PreEducation_Overig" in preEducation_dummies.columns:
            preEducation_dummies = preEducation_dummies.drop(["PreEducation_Overig"], axis=1)
            print('PreEducation_Overig')

        if "PreEducation_Wo" in preEducation_dummies.columns:
            preEducation_dummies = preEducation_dummies.drop(["PreEducation_Wo"], axis=1)
            print('PreEducation_Wo')         

        # Concatenate the dummy variables with the original DataFrame
        df_Clustering = pd.concat([df_Clustering, preEducation_dummies], axis=1)

        preEducations= ['PreEducation_Buitenlands', 'PreEducation_Hbo', 'PreEducation_Vwo']

        for col_name in preEducations:
            if col_name not in df_Clustering:
                df_Clustering[col_name] = 0
                
 
    # Drop the original 'Nationality' column
    df_Clustering = df_Clustering.drop(columns=["Nationality","PreEducation"])

    return df_Clustering


def PrepB1B2(df):
        # Assuming df is your DataFrame
    columns_to_drop = [col for col in df.columns if col.startswith(("3","4","5","6"))] + ["TheRest","OverAll"] + [col for col in df.columns if (col.startswith(("2")) and col.endswith("Endgrade") )]+ [col for col in df.columns if ("grade" in col.lower() )]
    #columns_to_drop += [col for col in df.columns if col.startswith("2")]  
    df = df.drop(columns=columns_to_drop)
    return df



In [ ]:
df = pd.read_excel(f'PrepareDatasets/Train_Program3_21_22.xlsx', sheet_name=0)
df = RemoveEntries(df)

In [ ]:
df["BSA"].value_counts()

In [ ]:
df_cluser = PrepCluster(df)
df_cluser = PrepB1B2(df_cluser)

In [ ]:
CheckNan(df_cluser)

In [ ]:
df_cluser["BSA"] = (df_cluser["Credits-Y1"] > 42).astype(int)

In [ ]:
df_cluser

# Model training

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score,recall_score, f1_score
import umap
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
df_cluser.drop(["Credits-Y1","Gender", 'PreEducation_Buitenlands',
       'PreEducation_Hbo', 'PreEducation_Vwo'] + [col for col in df_cluser.columns if col.endswith("_standardized")], axis=1).dropna().reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
reducer = umap.UMAP()
umap_embeddings_DESC = reducer.fit_transform(df_cluser.drop(["Credits-Y1","Gender", 'PreEducation_Buitenlands',
       'PreEducation_Hbo', 'PreEducation_Vwo'] + [col for col in df_cluser.columns if col.endswith("_standardized")], axis=1).dropna().reset_index(drop=True))


In [ ]:
def Plot2(df,cluster_labels,umap_embeddings,  ConfusionMatrix = True, color_dict= {}):

    hover_columns = ['Credits-Y1', "Gender", 'Nationality',"Crd-B1B2" ,"PreEducation","train", "BSA", "B1B2","TheRest","OverAll"] # 

    # Create hover text by concatenating values from specified columns
    hover_text = df.apply(lambda row: '<br>'.join(f"{col}: {row[col]}" for col in hover_columns), axis=1)
    len(hover_text)
    # Define marker symbols based on BSA values
    marker_symbols = ['square' if bsa == "PS" else 'cross' for bsa in df['BSA']]

    # marker_symbols[df[df["train"] == 20001521].index[0]] = "pentagon" 
    # marker_symbols[df[df["train"] == 20001585].index[0]] = "pentagon" 

    colors = cluster_labels.map(color_dict)

    # Create a scatter plot trace with hover data
    scatter_trace = go.Scatter(
        x=umap_embeddings[:, 0],
        y=umap_embeddings[:, 1],
        mode='markers',
        marker=dict(
            color=colors,
            size=10,
            symbol=marker_symbols,  # Set the marker symbols
            colorbar=dict(title='Cluster')
        ),
        text=hover_text  # Set the hover text
    )

    # Create a layout
    layout = go.Layout(
        title='UMAP 2-D representation of document embeddings with K-Means clustering',
        xaxis=dict(title='UMAP Dimension 1'),
        yaxis=dict(title='UMAP Dimension 2'),
        width=1600,  # Set width of the figure
        height=1200,  # Set height of the figure

    )

    # Create a figure
    fig = go.Figure(data=[scatter_trace], layout=layout)

    # Show the interactive plot
    fig.show()

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(2)
kmeans.fit(df_cluser.drop(["Credits-Y1","Gender", 'PreEducation_Buitenlands',
       'PreEducation_Hbo', 'PreEducation_Vwo'] + [col for col in df_cluser.columns if col.endswith("_standardized")], axis=1).dropna().reset_index(drop=True))
print(type(kmeans.labels_))

In [ ]:

# Plot2(
#     df,
#     pd.Series(kmeans.labels_),
#     umap_embeddings_DESC,
#     False,
#     {30: "Purple",
# 24:"Green",
# 18:"LightGreen",
# 2:"Green",
# 1:"LightGreen",
# 0:"blue"}
# )



Plot2(
    df,
    df["Crd-B1B2"],
    umap_embeddings_DESC,
    False,
    {30: "Purple",
24:"Green",
18:"LightGreen",
12:"Orange",
6:"Red",
0:"Black"}
)
